In [1]:
import xgboost as xgb
import pandas as pd
import warnings

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import ElasticNet

# import lightgbm as lgb
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso
from sklearn.linear_model import BayesianRidge
from sklearn.svm import SVR

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost as xgb


# Disable all warnings
warnings.filterwarnings('ignore')

# import models and fit

#### Enhanced EDA

In [2]:
df_final = pd.read_csv('../data/processed/df_final_processed_no_bins.csv')

In [3]:
df_final_ohe = pd.get_dummies(df_final, columns=['type', 'city'])

df_final_ohe

,year_built,sqft,baths,stories,beds,garage,is_foreclosure,price,basement,central_air,...,city_Salem,city_Sandy Springs,city_Shoreview,city_Springfield,city_Tallahassee,city_Tolleson,city_Trenton,city_Tumwater,city_Verona,city_Yukon
0,1990,1500,2.0,1.0,3.0,2.0,False,200000,0,1,...,False,False,False,False,False,False,False,False,False,False
1,2000,2000,3.0,2.0,3.0,2.0,False,200000,0,1,...,False,False,False,False,False,False,False,False,False,False
2,2000,2000,2.0,2.0,3.0,2.0,False,200000,0,1,...,False,False,False,False,False,False,False,False,False,False
3,1980,500,2.0,2.0,2.0,1.0,False,0,0,1,...,False,False,False,False,False,False,False,False,False,False
4,2000,3000,3.0,2.0,4.0,2.0,False,200000,0,0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4833,1980,1000,3.0,2.0,3.0,2.0,False,200000,1,1,...,False,False,False,False,False,False,False,False,False,False
4834,1980,4500,4.0,4.0,4.0,2.0,False,600000,1,1,...,False,False,False,False,False,False,False,False,False,False
4835,1990,2000,3.0,2.0,3.0,2.0,False,400000,1,1,...,False,False,False,False,False,False,False,False,False,False
4836,1890,2500,3.0,3.0,3.0,0.0,False,200000,1,1,...,False,False,False,False,False,False,False,False,False,False


In [4]:
# Convert boolean features to integers if necessary
df_final_ohe['is_foreclosure'] = df_final_ohe['is_foreclosure'].astype(int)
# Select columns that start with 'city_' or 'type_'
city_type_columns = df_final_ohe.filter(regex='^(city_|type_)').columns

# Convert these columns to integers
df_final_ohe[city_type_columns] = df_final_ohe[city_type_columns].astype(
    int)

In [5]:
df_final_ohe

,year_built,sqft,baths,stories,beds,garage,is_foreclosure,price,basement,central_air,...,city_Salem,city_Sandy Springs,city_Shoreview,city_Springfield,city_Tallahassee,city_Tolleson,city_Trenton,city_Tumwater,city_Verona,city_Yukon
0,1990,1500,2.0,1.0,3.0,2.0,0,200000,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2000,2000,3.0,2.0,3.0,2.0,0,200000,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2000,2000,2.0,2.0,3.0,2.0,0,200000,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1980,500,2.0,2.0,2.0,1.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,2000,3000,3.0,2.0,4.0,2.0,0,200000,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4833,1980,1000,3.0,2.0,3.0,2.0,0,200000,1,1,...,0,0,0,0,0,0,0,0,0,0
4834,1980,4500,4.0,4.0,4.0,2.0,0,600000,1,1,...,0,0,0,0,0,0,0,0,0,0
4835,1990,2000,3.0,2.0,3.0,2.0,0,400000,1,1,...,0,0,0,0,0,0,0,0,0,0
4836,1890,2500,3.0,3.0,3.0,0.0,0,200000,1,1,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Export processed dataframe after OHE to CSV
df_final_ohe.to_csv(
    '../data/processed/df_final_processed_OHE.csv', index=False)

#### Model Selection

In [7]:
# drop price column and do train test split
X = df_final_ohe.drop('price', axis=1)
y = df_final_ohe['price']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [8]:
# Exporting test/training data into CSV

X_train.to_csv('../data/training/X_train_v2_ohe.csv', index=False)
X_test.to_csv('../data/testing/X_test_v2_ohe.csv', index=False)
y_train.to_csv('../data/training/y_train_v2_ohe.csv', index=False)
y_test.to_csv('../data/testing/y_test_v2_ohe.csv', index=False)

In [9]:
# Define models
# Define models
models = {
    'Linear Regression': LinearRegression(),
    'Support Vector Machine': SVR(),
    'Random Forest': RandomForestRegressor(),
    'XGBoost': xgb.XGBRegressor(),
    'Decision Tree': DecisionTreeRegressor(),
    'K-Nearest Neighbors': KNeighborsRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'AdaBoost': AdaBoostRegressor(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'ElasticNet Regression': ElasticNet()
}

In [10]:
# Function to evaluate model
def evaluate_model(name, model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f'{name} Performance:')
    print(f'Mean Squared Error: {mse}')
    print(f'Mean Absolute Error: {mae}')
    print(f'R^2 Score: {r2}')
    print('')


# Evaluate each model
for name, model in models.items():
    evaluate_model(name, model, X_train, X_test, y_train, y_test)

Linear Regression Performance:
Mean Squared Error: 185504913290.16757
Mean Absolute Error: 183278.20756644657
R^2 Score: 0.4392068230097782

Support Vector Machine Performance:
Mean Squared Error: 346285720808.7064
Mean Absolute Error: 220627.45328381535
R^2 Score: -0.04684380631419338

Random Forest Performance:
Mean Squared Error: 1440394712.4304268
Mean Absolute Error: 12704.840613931523
R^2 Score: 0.9956455949733242

XGBoost Performance:
Mean Squared Error: 2504522904.0866137
Mean Absolute Error: 30133.225119850853
R^2 Score: 0.9924286676222395

Decision Tree Performance:
Mean Squared Error: 1157024793.3884296
Mean Absolute Error: 3305.785123966942
R^2 Score: 0.996502240300634

K-Nearest Neighbors Performance:
Mean Squared Error: 26662809917.355373
Mean Absolute Error: 70867.76859504133
R^2 Score: 0.9193966261278933

Gradient Boosting Performance:
Mean Squared Error: 24889542703.975716
Mean Absolute Error: 105212.73533476531
R^2 Score: 0.9247573259422873

AdaBoost Performance:
Mean

### Model Performance Comparison

Based on the performance metrics selected, the **Decision Tree Regressor** performed the best across all the models. See details results and comparison below, along with summary and next steps for Part 3 of the project:

#### Best Performing Model: Decision Tree Regressor
- **Mean Squared Error (MSE)**: 495,867,768.60
- **Mean Absolute Error (MAE)**: 2,479.34
- **R² Score**: 0.9985

#### Comparison with Other Models

| Model                         | Mean Squared Error (MSE)  | Mean Absolute Error (MAE)  | R² Score  |
|-------------------------------|---------------------------|----------------------------|-----------|
| **Decision Tree**             | 495,867,768.60            | 2,479.34                   | 0.9985    |
| **Random Forest**             | 1,024,435,105.73          | 11,535.48                  | 0.9969    |
| **XGBoost**                   | 2,504,522,904.09          | 30,133.23                  | 0.9924    |
| **Linear Regression**         | 185,504,913,290.17        | 183,278.21                 | 0.4392    |
| **Support Vector Machine**    | 346,285,720,808.71        | 220,627.45                 | -0.0468   |
| **K-Nearest Neighbors**       | 26,662,809,917.36         | 70,867.77                  | 0.9194    |
| **Gradient Boosting**         | 24,843,589,621.09         | 105,160.04                 | 0.9249    |
| **AdaBoost**                  | 105,503,589,057.43        | 281,912.39                 | 0.6811    |
| **Ridge Regression**          | 185,530,659,977.92        | 183,475.83                 | 0.4391    |
| **Lasso Regression**          | 185,503,888,825.00        | 183,296.20                 | 0.4392    |
| **ElasticNet Regression**     | 234,388,902,656.75        | 189,973.07                 | 0.2914    |

### Summary

The **Decision Tree Regressor** achieved the best performance in terms of Mean Squared Error, Mean Absolute Error, and R² Score, indicating that it is the most accurate model among those tested for this particular dataset.

The **Random Forest Regressor** also performed very well, closely following the Decision Tree Regressor. The other models, especially the simpler linear models (Linear Regression, Ridge, Lasso, and ElasticNet), did not perform as well, which might suggest that the dataset benefits from more complex, non-linear models.

Based on these results, you might consider focusing on the Decision Tree and Random Forest models for further tuning and optimization.


### Next Steps for Part 3
Based on these results, we'll focusing on the Decision Tree and Random Forest models for further tuning and optimization.